In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers.legacy import Adam, RMSprop

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
main_dir = os.environ.get("MAIN_DIRECTORY")
os.chdir(main_dir)

In [ ]:
model_save_path = os.environ.get("PRETRAINED_MODELS_PATH")

In [ ]:
sentiments = ['negative', 'neutral', 'positive']
print(sentiments)

In [ ]:
train_img_cnt = []

for sent in sentiments:
    img_cnt = len(os.listdir('models/video_model/data/train/'+sent))
    print(f"No. of train '{sent}' images: {img_cnt}")
    train_img_cnt.append(img_cnt)

In [ ]:
test_img_cnt = []

for sent in sentiments:
    img_cnt = len(os.listdir('models/video_model/data/test/'+sent))
    print(f"No. of test '{sent}' images: {img_cnt}")
    test_img_cnt.append(img_cnt)

In [ ]:
colors = ['r', 'gray', 'g']
fig, ax = plt.subplots(1,2,figsize=(15,5))

ax[0].bar(sentiments, train_img_cnt, color=colors)
ax[0].set_title('No. of train images')

ax[1].bar(sentiments, test_img_cnt, color=colors)
ax[1].set_title('No. of test images')

plt.tight_layout()
plt.show()

In [ ]:
X, y = [], []

for sent in sentiments:
    path = os.path.join("models/video_model/data/train", sent)
    label = sentiments.index(sent)
    for filename in os.listdir(path):
        img_path = os.path.join(path, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (48, 48))
        X.append(img)
        y.append(label)

X = np.array(X)
y = np.array(y)

In [ ]:
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

In [ ]:
plt.figure(figsize=(12,5))
for i in range(1,6):
    plt.subplot(1,5,i)
    rand_num = np.random.randint(0,24612 + 1)
    plt.imshow(X[rand_num])
    plt.title(f"Sentiment: {sentiments[y[rand_num]].capitalize()}")
    plt.tight_layout()
    plt.axis('off')
plt.show()

In [ ]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

In [ ]:
train_gen = datagen.flow_from_directory(
    "models/video_model/data/train",
    target_size = (48, 48),
    batch_size = 32,
    class_mode = 'categorical'
)

validation_gen = datagen.flow_from_directory(
    "models/video_model/data/test",
    target_size = (48, 48),
    batch_size = 32,
    class_mode = 'categorical'
)

In [ ]:
model = Sequential([
                Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 3),padding='same'),
                Conv2D(64, kernel_size=(3, 3), activation='relu'),
                MaxPooling2D(pool_size=(2, 2)),
                Dropout(0.25),

                Conv2D(64, kernel_size=(3, 3), activation='relu'),
                MaxPooling2D(pool_size=(2, 2)),
                Conv2D(128, kernel_size=(3, 3), activation='relu'),
                MaxPooling2D(pool_size=(2, 2)),
                Dropout(0.25),

                Flatten(),
                Dense(64, activation='relu'),
                Dropout(0.5),
                Dense(3, activation='softmax')]
                   )

In [ ]:
model.summary()

In [ ]:
opt = Adam(learning_rate=0.0001,decay=1e-6)

model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
history = model.fit(
        train_gen,
        steps_per_epoch= 24612 // 64,
        epochs = 50,
        validation_data = validation_gen,
        validation_steps = 6154 // 64
        )

In [ ]:
model_json = model.to_json()
with open('/content/video_model.json', 'w') as json_file:
    json_file.write(model_json)